# 🍿 Content-Based Aanbevelingen

In dit notebook laten we snel zien hoe je een **content-based** aanbevelingssysteem kunt opzetten. Bij content-based filtering worden aanbevelingen gedaan op basis van de eigenschappen van items, in dit geval films. We gebruiken de kenmerken zoals `Genre`, `Actors`, `Plot`, `Title`, `Director`, en `Country` om de overeenkomst tussen films te berekenen. Deze aanpak is nuttig voor aanbevelingen waarbij de inhoud van een film overeenkomt met films die een gebruiker al leuk vindt.

### Hoe werkt het?
Het systeem berekent de overeenkomst tussen films door een **TF-IDF vectorizer** te gebruiken om woorden in de kenmerken te transformeren naar numerieke representaties, gevolgd door het berekenen van **cosine similarity** om de overeenkomst tussen films te meten. Dit betekent dat films met vergelijkbare genres, acteurs of verhaallijnen waarschijnlijk aan elkaar worden aanbevolen.

### Functie: `get_content_based_recommendations`
De functie `get_content_based_recommendations` maakt aanbevelingen voor een specifieke film gebaseerd op de inhoudskenmerken. Hier is een stap-voor-stap uitleg van hoe de functie werkt:

1. **Combineer Kenmerken**: De functie voegt `Plot`, `Title`, `Genre`, `Director`, en `Country` samen tot één enkele tekstuele feature. Dit maakt het eenvoudiger om overeenkomsten tussen films te berekenen.
2. **TF-IDF Vectorisatie**: Een TF-IDF vectorizer zet deze gecombineerde kenmerken om naar een numerieke representatie, waarbij veelvoorkomende woorden minder gewicht krijgen.
3. **Cosine Similarity Berekenen**: De functie berekent de cosine similarity tussen de geselecteerde film en alle andere films in de dataset.
4. **Sorteren op Overeenkomst**: De films worden gesorteerd op basis van hun overeenkomst met de geselecteerde film, en de top N meest vergelijkbare films worden geselecteerd (behalve de gekozen film zelf).
5. **Aanbevelingen Teruggeven**: Uiteindelijk geeft de functie een DataFrame terug met aanbevolen films.

### Voorbeeld Code
Hieronder zie je de code die de content-based aanbevelingen maakt:


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

df_movies = pd.read_csv('data/movies.csv')


def get_content_based_recommendations(movie_id, df, top_n=6):
    # Combine 'Genre' and 'Actors' into a single feature for similarity calculation
    df['combined_features'] = df['Plot'].fillna('') + ' ' + df['Title'].fillna('') + ' ' + df['Genre'].fillna('') + ' ' + df['Director'].fillna('') + ' ' + df['Country'].fillna('')
    
    # Initialize the TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    
    # Fit and transform the combined features
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_features'])
    
    # Calculate cosine similarity matrix
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Get the index of the selected movie based on its movieId
    idx = df.index[df['movieId'] == movie_id].tolist()[0]
    
    # Get similarity scores for this movie, sorted by similarity
    similarity_scores = list(enumerate(cosine_sim[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Get indices of the most similar movies (excluding the selected movie itself)
    similar_movie_indices = [i[0] for i in similarity_scores[1:top_n + 1]]
    
    # Return DataFrame of recommended movies
    return df.iloc[similar_movie_indices]

In [2]:
df_movies[df_movies['movieId'] == 3114]

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,DVD,BoxOffice,Production,Website,Response,Season,Episode,seriesID,totalSeasons,movieId
515,515,Toy Story 2,1999,G,24 Nov 1999,92 min,"Animation, Adventure, Comedy","John Lasseter, Ash Brannon, Lee Unkrich","John Lasseter, Pete Docter, Ash Brannon","Tom Hanks, Tim Allen, Joan Cusack",...,NaN,"$245,852,179",NaN,NaN,True,NaN,NaN,NaN,NaN,3114.0


In [3]:
get_content_based_recommendations(3114, df_movies, top_n=6)

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,BoxOffice,Production,Website,Response,Season,Episode,seriesID,totalSeasons,movieId,combined_features
4855,4855,Toy Story,1995,G,22 Nov 1995,81 min,"Animation, Adventure, Comedy",John Lasseter,"John Lasseter, Pete Docter, Andrew Stanton","Tom Hanks, Tim Allen, Don Rickles",...,"$223,225,679",NaN,NaN,True,NaN,NaN,NaN,NaN,1.0,A little boy named Andy loves to be in his roo...
6814,6814,Toy Story 3,2010,G,29 Jun 2010,103 min,"Animation, Adventure, Comedy",Lee Unkrich,"John Lasseter, Andrew Stanton, Lee Unkrich","Tom Hanks, Tim Allen, Joan Cusack",...,"$415,004,880",NaN,NaN,True,NaN,NaN,NaN,NaN,78499.0,"Woody, Buzz and the whole gang are back. As th..."
2294,2294,Fullmetal Alchemist the Movie: Conqueror of Sh...,2005,PG-13,25 Aug 2006,105 min,"Animation, Action, Adventure",Seiji Mizushima,"Hiromu Arakawa, Shô Aikawa, Jared Hedges","Vic Mignogna, Aaron Dismuke, Jason Liebrecht",...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,68959.0,"Edward Elric, the Fullmetal Alchemist, is on t..."
3970,3970,Hot Shots! Part Deux,1993,PG-13,21 May 1993,86 min,"Action, Comedy",Jim Abrahams,"Jim Abrahams, Pat Proft","Charlie Sheen, Lloyd Bridges, Valeria Golino",...,"$38,922,972",NaN,NaN,True,NaN,NaN,NaN,NaN,466.0,Topper Harley is found working as an odd-job-m...
5429,5429,Home Alone 3,1997,PG,12 Dec 1997,102 min,"Action, Comedy, Crime",Raja Gosnell,John Hughes,"Alex D. Linz, Olek Krupa, Rya Kihlstedt",...,"$30,882,515",NaN,NaN,True,NaN,NaN,NaN,NaN,1707.0,"Four high-tech industrial spies, Beaupre, Alic..."
8998,8998,Down to You,2000,PG-13,21 Jan 2000,91 min,"Comedy, Drama, Romance",Kris Isacsson,Kris Isacsson,"Freddie Prinze Jr., Julia Stiles, Selma Blair",...,"$20,069,008",NaN,NaN,True,NaN,NaN,NaN,NaN,3225.0,This light-hearted romantic comedy centers tel...
